<a href="https://colab.research.google.com/github/Transega/Forest-canopy-density-Model-/blob/main/TrofmiS_fcd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs geemap package
import subprocess
 
try:
    import geemap
    import geopandas as gpd
    import numpy as np
    import folium
    import rasterio
    import tensorflow
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])
    subprocess.check_call(["python", '-m', 'pip', 'install', 'folium'])
    subprocess.check_call(["python", '-m', 'pip', 'install', "rasterio"])
    subprocess.check_call(["python", '-m', 'pip', 'install', "tensorflow"])
 
# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap
 
# Authenticates and initializes Earth Engine
import ee
 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=tXi2U5FePpkH8M2SAJ2C-7hLXGy4yAMR4pru32ouOx0&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgGLkr-MMLoCbtr6h66avQTf2An_oAuJ6n9abOgXFThs6Vl73421wc

Successfully saved authorization token.


In [ ]:
# Uganda_Sentinel = ee.Image("users/collinsasegaca/FCD_UGANDA_2016_2020_final")

In [ ]:
import ee
import geemap
import geemap.eefolium
import rasterio as rio
from rasterio.plot import show
from sklearn import cluster
import matplotlib.pyplot as plt
import matplotlib.colors as mc
import numpy as np
import cv2
from PIL import Image
import pandas as pd
import matplotlib.gridspec as grid
import tensorflow as tf
import numpy as np
import rasterio, rasterio.mask, fiona
import matplotlib.pyplot as plt
# from sentinelhub import WmsRequest, MimeType, CRS, BBox, CustomUrlParam, geo_utils, DataSource

In [ ]:
AOI = ee.FeatureCollection("users/snyawacha/FINAL_AOI_FOREST") #AOI = ee.FeatureCollection();


In [ ]:
Ke_priority = ee.FeatureCollection('users/collinsasegaca/Kenya_priority_forest')

In [ ]:
AOI = Ke_priority.filter(ee.Filter.eq('NAME', 'Kakamega'))

In [ ]:
Kenya = ee.FeatureCollection("users/collinsasegaca/Kenya").geometry()

In [ ]:
Map = geemap.Map()

# ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

l8collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                  .filter(ee.Filter.date('2020-01-01', '2021-12-31')) \
                  .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
                  .filterBounds(AOI)\



# NDVI function for L8
def NDVI_L8(image):
  ndvi= image.normalizedDifference(['B5','B4']).rename('NDVI') #Compute the NDVI values while Using the NIR and Red Bands
  return ndvi
limage = l8collection.median().clip(AOI)
# Map.centerObject(AOI, 12)
# Map.addLayer(image, {}, 'image') 


In [ ]:
# Cloud masking function.
# Landsat 8

def maskL8sr_T1_L2(image):
  bands = ["SR_B1","SR_B2","SR_B3","SR_B4","SR_B5","SR_B6","SR_B7","ST_B10",]
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

# Sentinel 2  

def maskS2clouds(image): 
   qa = image.select('QA60');

   # Bits 10 and 11 are clouds and cirrus, respectively.
   cloudBitMask = 1 << 10;
   cirrusBitMask = 1 << 11;

   # Both flags should be set to zero, indicating clear conditions.
   mask = qa.bitwiseAnd(cloudBitMask).eq(0)#and(qa.bitwiseAnd(cirrusBitMask).eq(0));

   return image.updateMask(mask).divide(10000);

In [ ]:
def maskL8sr(image):
  bands = ["B1","B2","B3","B4","B5","B6","B7","B10","B11"]
  cloudShadowBitMask = ee.Number(2).pow(3).int()
  cloudsBitMask = ee.Number(2).pow(5).int()
  qa = image.select('BQA')
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask).select(bands).divide(10000)

In [ ]:
image = l8collection.map(maskL8sr_T1_L2).median().clip(AOI)

In [ ]:
image.reduceRegion(ee.Reducer.minMax(), AOI, scale= 30).getInfo()  #ee.Reducer.mean(), roi, scale= 300)

{'SR_B1_max': 1.026900053024292,
 'SR_B1_min': 0.710099995136261,
 'SR_B2_max': 1.0787999629974365,
 'SR_B2_min': 0.7398999929428101,
 'SR_B3_max': 1.2281500101089478,
 'SR_B3_min': 0.7936000227928162,
 'SR_B4_max': 1.3453999757766724,
 'SR_B4_min': 0.7749999761581421,
 'SR_B5_max': 2.405250072479248,
 'SR_B5_min': 0.9943000078201294,
 'SR_B6_max': 2.0183000564575195,
 'SR_B6_min': 0.8496500253677368,
 'SR_B7_max': 1.679800033569336,
 'SR_B7_min': 0.7947999835014343,
 'ST_B10_max': 4.65749979019165,
 'ST_B10_min': 4.315450191497803}

In [ ]:
s2 = ee.ImageCollection("COPERNICUS/S2") \
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5)) \
        .sort('CLOUDY_PIXEL_PERCENTAGE') \
        .filterBounds(AOI) \
        .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
        

s2_image = s2.map(maskS2clouds).median().clip(AOI)

IndentationError: ignored

In [ ]:
s2_image.reduceRegion(ee.Reducer.minMax(), AOI, scale= 100).getInfo()

In [ ]:
M = geemap.Map()
M.addLayer(image,{'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min':-1,'max':2 },'Image')
M.addLayer(s2_image, {'bands': ['B4', 'B3', 'B2'], 'min':0,'max':10},'s2')
M.centerObject(image, 12)
M

normalization function

In [ ]:
def normalization_100(image,region,scale):
  mean_std = image.reduceRegion( ee.Reducer.mean()
            .combine(ee.Reducer.stdDev(),None, True),region,
  scale= scale,
  maxPixels = 10e9,
  tileScale= 16)
  
  def unit_(name):
      name = ee.String(name);
      band = image.select(name);
      mean=ee.Number(mean_std.get(name.cat('_mean')));
      std=ee.Number(mean_std.get(name.cat('_stdDev')));
      max=mean.add(std.multiply(3))
      min=mean.subtract(std.multiply(3))
      band1=ee.Image(min).multiply(band.lt(min)).add(ee.Image(max).multiply(band.gt(max))).add(band.multiply(ee.Image(1).subtract(band.lt(min)).subtract(band.gt(max))))
      result_band=band1.subtract(min).divide(max.subtract(min))#.rename(image.bandNames());
      return result_band#;

  #  mean_value = Image.select(band).reduceRegion(
    # ee.Reducer.mean(), roi, scale= 300).get(band).getInfo()
  def nom_value(image): 
    return image.multiply(100)
    

# // use unit scale to normalize the pixel values
  unitScale = ee.ImageCollection.fromImages(image.bandNames().map(unit_)).toBands().rename(image.bandNames())
  
  return unitScale.multiply(100)


In [ ]:
#+++++++ +++++  Advanced Vegetation index +++++++ +++++ \   bands ['B2','B3','B4', 'B5', 'B6'], ['Green','Blue','RED', 'NIR','SWIR1']
def AVI_func(image, AOI, scale):
  # image = image.clip(roi)
  AVI = image.expression('((nir + 1 )* (65536-red) *(nir -red))**0.333',{     
  'nir':image.select(['NIR']),    
  'red':image.select(['RED'])
  }
).rename('AVI')

  AVI_norm = normalization_100(AVI,AOI,scale)
  return AVI_norm

#  +++++++ +++++ Bare Soil Index +++++++ \

def BSI_func(image):
  # image = image.clip(roi)
  BSI = image.expression('((SWIR1 + red) - (red+ G) / (SWIR1 + red) + (red+ G))*100-100',{
  'SWIR1':image.select(['SWIR1']),
  'G':image.select(['Green']),
  'red':image.select(['RED'])
  }
).rename('BSI')

  return BSI

# Soil index
def SI_func(image, AOI, scale):
  # image = image.clip(roi)
  SI = image.expression('((65536-Green)*(65536-Blue)*(65536-Red))',{
  'Red':image.select(['RED']),
  'Green':image.select(['Green']), #******
  'Blue':image.select(['Blue'])
  }
).rename('SSI')

  SSI = normalization_100(SI,AOI,scale)
 #'(((256-Blue) * (256-Green) * (256-Red))/3   (((1-Red)*(1-Green)*(1-Blue))0.33)#
  return SSI

In [ ]:
# fcd.reduceRegion(ee.Reducer.minMax(), AOI, scale = 30 ).getInfo() 

In [ ]:
vis_params = {
  'min': 1,
  'max': 90,
  'palette': [
    'ff4c16', 'ffd96c', '39a71d','blue'
  ],
}

In [ ]:
def FCD_computation_Landsat(AOI, year,scale):
  # if (image_collection == )
  l8collection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
                  .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \
                  .select(['B2','B3','B4', 'B5', 'B6'], ['Green','Blue','RED', 'NIR','SWIR1'])\
                  .filter(ee.Filter.lt('CLOUD_COVER', 40)) \
                  .filterBounds(AOI)\
                  
#  ["SR_B1","SR_B2","SR_B3","SR_B4","SR_B5","SR_B6","SR_B7","ST_B10",]

  L8_image = l8collection.median().clip(AOI)
  # print(L8_image.bandNames().getInfo())

  #compute indices 
  AVI_raw = AVI_func(L8_image,AOI,scale) 
  # BSI_raw = BSI_func(L8_image)
  SI_raw = SI_func(L8_image,AOI,scale)
  
  # Normalize data
  # AVI_out = normalization_100(AVI_raw,AOI,scale)
  # # BSI = normalization_100(bsi,AOI,scale)
  # SSI_out = normalization_100(SI_raw,AOI,scale).rename('SSI')

  # print(SSI_out.bandNames().getInfo())

  FCD_output = AVI_raw.expression(
                '(AVI*SSI+1-1)**0.5',
                {
                  'AVI': AVI_raw.select('AVI'),
                  'SSI': SI_raw.select('SSI')
                }).rename('FCD')

  FCD_norm = normalization_100(FCD_output,AOI,scale) 
  return FCD_output


In [ ]:
def FCD_computation_S2_1(AOI, year,scale):
  # if (image_collection == )
  S2collection = ee.ImageCollection("COPERNICUS/S2") \
    .filterBounds(AOI) \
    .select(['B3','B2','B4','B8','B11'],['Green','Blue','RED', 'NIR','SWIR1']) \
    .filter(ee.Filter.date('2020-01-01', '2020-12-31')) \


  S2_image = S2collection.median().clip(AOI)
  print(S2_image.bandNames().getInfo())

  #compute indices 
  AVI_raw = AVI_func(S2_image,AOI,scale)
  # BSI_raw = BSI_func(S2_image)
  SI_raw = SI_func(S2_image,AOI,scale)
  
  # Normalize data
  # AVI_out = normalization_100(AVI_raw,AOI,scale)
  # # BSI = normalization_100(bsi,AOI,scale)
  # SSI_out = normalization_100(SI_raw,AOI,scale).rename('SSI')

  print(SI_raw.bandNames().getInfo())

  FCD_output = AVI_raw.expression(
                '((AVI*SSI+1)**0.5)-1',
                {
                  'AVI': AVI_raw.select('AVI'),
                  'SSI': SI_raw.select('SSI')
                }).rename('FCD')

        

  # FCD_norm = normalization_100(FCD_output,AOI,scale) 
  return FCD_output

In [ ]:
test_2 = FCD_computation_S2_1(AOI, "2020",10)

['Green', 'Blue', 'RED', 'NIR', 'SWIR1']
['SSI']


In [ ]:
# test.bandNames().getInfo()

In [ ]:
test = FCD_computation_Landsat(Kenya, '2016', 30)

In [ ]:
ke = normalization_100(test,Kenya,scale)

In [ ]:
kak = test.clip(AOI)

In [ ]:
Map = geemap.Map()
Map.centerObject(AOI, 12)
# Map.addLayer(image, {}, 'image',0) 
Map.setOptions('SATELLITE')
Map.addLayer(test_2 , vis_params, 'FCD')
Map

In [ ]:
url = test_2.getDownloadUrl({
				'scale': 30, 			
				'crs': 'EPSG:4326', 
				'region': AOI.geometry(),
        'maxPixels': 1e19
			}) 

In [ ]:
url

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/d7db004ff7197d6f00c8ef482ebd3f50-116bf72cac448df366ce01cff5636560:getPixels'

Cloud masking

In [ ]:

CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50


In [ ]:
def get_s2_sr_cld_col(aoi, start_date, end_date):

    

    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)
def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw).divide(10000)

In [ ]:
def FCD_computation_S2(AOI, year,scale):
  # if (image_collection == )
  # obtain the image collection 
  s2_sr_cld_col = get_s2_sr_cld_col(AOI, year+'-01-01', year+'-12-31')

  S2_image = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .select(['B3','B2','B4','B8','B11'],['Green','Blue','RED', 'NIR','SWIR1'])
                             .median().clip(AOI))


  # S2_image = S2collection.median().clip(AOI)
  print(S2_image.bandNames().getInfo())

  #compute indices 
  AVI_raw = AVI_func(S2_image,AOI,scale)
  # BSI_raw = BSI_func(S2_image)
  SI_raw = SI_func(S2_image,AOI,scale)
  
  # Normalize data
  # AVI_out = normalization_100(AVI_raw,AOI,scale)
  # # BSI = normalization_100(bsi,AOI,scale)
  # SSI_out = normalization_100(SI_raw,AOI,scale).rename('SSI')

  print(SI_raw.bandNames().getInfo())

  FCD_output = AVI_raw.expression(
                '((AVI*SSI+1)**0.5)-1',
                {
                  'AVI': AVI_raw.select('AVI'),
                  'SSI': SI_raw.select('SSI')
                }).rename('FCD')

  # FCD_norm = normalization_100(FCD_output,AOI,scale) 
  return FCD_output

In [ ]:
roi = Ke_priority.filter(ee.Filter.eq('NAME', 'Kakamega')).geometry()

In [ ]:
fcd_text = FCD_computation_S2(Kenya, '2020', 10)

['Green', 'Blue', 'RED', 'NIR', 'SWIR1']


EEException: ignored

In [ ]:
M = geemap.Map()
# M.addLayer(s2_sr_median,{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
#                 'S2 cloud-free mosaic', 'clodles')
M.addLayer(fcd_text, vis_params, 'FCD')
M.centerObject(AOI, 6)

EEException: ignored

In [ ]:
# task.start()

In [ ]:
# task.status()

In [ ]:
M

In [ ]:
# fcd_text.bandNames().getInfo()

In [ ]:
M

In [ ]:
# geemap.ee_export_image(kak, filename='KeFCD2020.tif',scale=30, region= Kenya,file_per_band=True)